# Integrasi dari Spark ke Apache Superset

Notebook ini menjelaskan cara mengintegrasikan analisis data kartu kredit dari Spark ke Apache Superset untuk visualisasi interaktif dan pembuatan dashboard.

## 1. Overview Arsitektur Integrasi

Arsitektur sistem untuk analisis data kartu kredit terdiri dari beberapa komponen:

1. **HDFS (Hadoop Distributed File System)**: Menyimpan data kartu kredit mentah dalam format CSV
2. **Apache Spark**: Memproses dan menganalisis data, serta menyimpan hasil ke Hive
3. **Apache Hive**: Sebagai data warehouse yang menjembatani Spark dan Superset
4. **Apache Superset**: Visualisasi dan dashboard interaktif

Alur data:
- Data CSV disimpan di HDFS
- Spark memproses data dan menghasilkan insight
- Data dimasukkan ke Hive untuk akses SQL
- Superset mengakses Hive untuk visualisasi

## 2. Penyiapan Data untuk Superset

Superset membutuhkan akses ke data yang dapat diquery melalui SQL. Untuk itu, kita perlu memastikan data sudah tersedia di Hive dan terstruktur dengan baik.

In [ ]:
# Import library yang dibutuhkan
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Membuat Spark Session dengan dukungan untuk Hive
spark = SparkSession.builder \
    .appName("Prepare Data for Superset") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

print(f"Spark version: {spark.version}")
print(f"Spark master: {spark.sparkContext.master}")

In [ ]:
# Memastikan data sudah di HDFS
try:
    # Check if data is in HDFS
    check_cmd = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem.get(
        spark.sparkContext._jsc.hadoopConfiguration()
    ).exists(
        spark.sparkContext._jvm.org.apache.hadoop.fs.Path("/data/credit-card-data/DataFixABD.csv")
    )
    
    if check_cmd:
        print("Data file exists in HDFS")
    else:
        print("Data file not found in HDFS, will try to upload it")
        # You can add code here to upload the file if needed
except Exception as e:
    print(f"Error checking HDFS: {str(e)}")

# Define schema
schema = StructType([
    StructField("Tanggal", StringType(), True),
    StructField("Provinsi", StringType(), True),
    StructField("Nilai_Server", DoubleType(), True),
    StructField("Nilai_Kartu", DoubleType(), True),
    StructField("Nilai_Registered", DoubleType(), True),
    StructField("Nilai_Unregistered", DoubleType(), True)
])

# Try to load from HDFS
try:
    df = spark.read.csv("hdfs:///data/credit-card-data/DataFixABD.csv", header=True, schema=schema)
    print(f"Successfully loaded data from HDFS, row count: {df.count()}")
except Exception as e:
    print(f"Error: {str(e)}")
    print("Trying local file...")
    
    # Try local file
    try:
        df = spark.read.csv("/home/jovyan/data/DataFixABD.csv", header=True, schema=schema)
        print(f"Successfully loaded data from local file, row count: {df.count()}")
    except Exception as e2:
        print(f"Error with local file: {str(e2)}")
        raise Exception("Could not load data from any location")

In [ ]:
# Process the data for Superset
df = df.withColumn("tanggal", to_date(col("Tanggal")))
df = df.withColumn("tahun", year(col("tanggal")))
df = df.withColumn("bulan", month(col("tanggal")))
df = df.withColumn("nama_bulan", date_format(col("tanggal"), "MMMM"))
df = df.withColumn("hari", dayofmonth(col("tanggal")))
df = df.withColumn("hari_dalam_minggu", date_format(col("tanggal"), "EEEE"))

# Display schema and sample data
print("Schema:")
df.printSchema()

print("\nSample data:")
df.show(5)

In [ ]:
# Create or replace Hive table
print("Creating/replacing Hive table 'credit_card_data'...")

# Drop table if exists to ensure clean state
spark.sql("DROP TABLE IF EXISTS credit_card_data")

# Create table with processed data
df.write.format("hive").mode("overwrite").saveAsTable("credit_card_data")

# Verify table creation
tables = spark.sql("SHOW TABLES").collect()
print("Available tables:")
for table in tables:
    print(table[0])

# Check table structure
print("\nTable structure:")
spark.sql("DESCRIBE credit_card_data").show()

# Check row count
row_count = spark.sql("SELECT COUNT(*) FROM credit_card_data").collect()[0][0]
print(f"\nTable has {row_count} rows")

## 3. Mempersiapkan Agregasi untuk Superset

Apache Superset akan lebih optimal jika data yang akan divisualisasikan sudah diaregasikan sesuai kebutuhan. Berikut beberapa agregasi dasar yang dapat membantu visualisasi di Superset:

In [ ]:
# 1. Agregasi bulanan untuk tren transaksi
print("Creating monthly aggregation view...")

spark.sql("""
CREATE OR REPLACE VIEW monthly_agg AS
SELECT 
    tahun, 
    bulan,
    nama_bulan,
    AVG(Nilai_Server) as avg_server,
    AVG(Nilai_Kartu) as avg_kartu,
    AVG(Nilai_Registered) as avg_registered,
    AVG(Nilai_Unregistered) as avg_unregistered,
    MAX(Nilai_Server) as max_server,
    MIN(Nilai_Server) as min_server,
    COUNT(*) as count_days
FROM credit_card_data
GROUP BY tahun, bulan, nama_bulan
ORDER BY tahun, bulan
""")

# Display the view
print("Sample data from monthly aggregation:")
spark.sql("SELECT * FROM monthly_agg LIMIT 5").show()

In [ ]:
# 2. Agregasi mingguan untuk pola hari dalam seminggu
print("Creating day of week aggregation view...")

spark.sql("""
CREATE OR REPLACE VIEW dayofweek_agg AS
SELECT 
    hari_dalam_minggu,
    AVG(Nilai_Server) as avg_server,
    AVG(Nilai_Kartu) as avg_kartu,
    AVG(Nilai_Registered) as avg_registered,
    AVG(Nilai_Unregistered) as avg_unregistered,
    COUNT(*) as count_occurrences
FROM credit_card_data
GROUP BY hari_dalam_minggu
ORDER BY 
    CASE 
        WHEN hari_dalam_minggu = 'Monday' THEN 1
        WHEN hari_dalam_minggu = 'Tuesday' THEN 2
        WHEN hari_dalam_minggu = 'Wednesday' THEN 3
        WHEN hari_dalam_minggu = 'Thursday' THEN 4
        WHEN hari_dalam_minggu = 'Friday' THEN 5
        WHEN hari_dalam_minggu = 'Saturday' THEN 6
        WHEN hari_dalam_minggu = 'Sunday' THEN 7
    END
""")

# Display the view
print("Sample data from day of week aggregation:")
spark.sql("SELECT * FROM dayofweek_agg").show()

In [ ]:
# 3. Mempersiapkan data untuk deteksi outlier
print("Creating outlier detection view...")

spark.sql("""
CREATE OR REPLACE VIEW outlier_detection AS
WITH stats AS (
    SELECT
        AVG(Nilai_Server) as mean_server,
        STDDEV(Nilai_Server) as stddev_server
    FROM credit_card_data
)
SELECT 
    a.tanggal,
    a.Provinsi,
    a.Nilai_Server,
    a.Nilai_Kartu,
    a.Nilai_Registered,
    a.Nilai_Unregistered,
    (a.Nilai_Server - b.mean_server) / b.stddev_server as z_score
FROM credit_card_data a
CROSS JOIN stats b
WHERE ABS((a.Nilai_Server - b.mean_server) / b.stddev_server) > 2
ORDER BY ABS((a.Nilai_Server - b.mean_server) / b.stddev_server) DESC
""")

# Display the view
print("Sample data from outlier detection view:")
spark.sql("SELECT * FROM outlier_detection LIMIT 10").show()

## 4. Koneksi dari Superset ke Hive

Untuk menghubungkan Superset dengan data di Hive, kita perlu mengkonfigurasi koneksi database di Superset. Hal ini dilakukan melalui UI Superset, namun kita juga dapat melakukannya secara otomatis melalui API atau saat startup container.

Berikut konfigurasi yang perlu diatur:
1. URI koneksi: `hive://hive@spark-master:10000/default`
2. Nama koneksi: `credit_card_db`

Setelah koneksi dibuat, kita dapat membuat dataset dari tabel dan view Hive yang telah kita buat.

## 5. Tipe Chart yang Sesuai untuk Data Kartu Kredit di Superset

Apache Superset menyediakan banyak jenis visualisasi yang dapat digunakan untuk analisis data kartu kredit. Berikut beberapa jenis visualisasi yang cocok:

1. **Line Chart**: Untuk menampilkan tren nilai transaksi dari waktu ke waktu
2. **Bar Chart**: Untuk membandingkan nilai transaksi berdasarkan hari dalam minggu
3. **Heatmap**: Untuk menunjukkan korelasi antar metrik transaksi
4. **Scatter Plot**: Untuk deteksi outlier, dengan nilai transaksi pada sumbu Y dan tanggal pada sumbu X
5. **Box Plot**: Untuk melihat distribusi nilai transaksi
6. **Bubble Chart**: Untuk analisis multi-dimensi, misalnya nilai server (ukuran), nilai kartu (warna), dan waktu (sumbu X)

Semua jenis visualisasi ini dapat dikombinasikan dalam dashboard Superset yang komprehensif.

## 6. Contoh Query SQL untuk Superset

Berikut beberapa contoh query SQL yang dapat digunakan di Superset untuk membuat visualisasi:

1. **Tren Bulanan**:
```sql
SELECT 
    tahun, 
    bulan,
    nama_bulan,
    avg_server,
    avg_kartu
FROM monthly_agg
ORDER BY tahun, bulan
```

2. **Pola Mingguan**:
```sql
SELECT 
    hari_dalam_minggu,
    avg_server,
    avg_kartu
FROM dayofweek_agg
```

3. **Deteksi Outlier**:
```sql
SELECT 
    tanggal,
    Nilai_Server,
    z_score
FROM outlier_detection
WHERE ABS(z_score) > 2
ORDER BY ABS(z_score) DESC
```

4. **Perbandingan Provinsi**:
```sql
SELECT 
    Provinsi,
    AVG(Nilai_Server) as avg_server,
    AVG(Nilai_Kartu) as avg_kartu
FROM credit_card_data
GROUP BY Provinsi
ORDER BY avg_server DESC
```

## 7. Rekomendasi Desain Dashboard

Berikut rekomendasi desain dashboard Superset untuk analisis data kartu kredit:

1. **Dashboard Overview**:
   - Metrik utama (total transaksi, rata-rata nilai, dll)
   - Tren bulanan dalam bentuk line chart
   - Breakdown harian dalam bentuk heatmap
   
2. **Dashboard Analisis Pola**:
   - Pola mingguan dalam bentuk bar chart
   - Distribusi nilai transaksi dalam bentuk histogram
   - Scatter plot untuk deteksi outlier
   
3. **Dashboard Analisis Regional**:
   - Perbandingan antar provinsi dalam bentuk bar chart
   - Peta choropleth untuk visualisasi geografis
   - Tren per provinsi dalam bentuk line chart multiseries

Setiap dashboard dapat dilengkapi dengan filter interaktif untuk memudahkan eksplorasi data.

In [ ]:
# Close the Spark session
spark.stop()
print("Integration setup complete!")